In [1]:
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine
from ta import add_all_ta_features


In [2]:
portfolio = ["AAPL", "DIS","MSFT", "GOOGL", "NVDA", "DHR", "AMD", "SPY"]

In [3]:
def download_data(stock):
    stock_data = yf.download(stock, start="2018-1-01", period="max")
    stock_data["Symbol"] = stock
    return stock_data


In [4]:
def initialize_DB():
    # Replace with your database connection details
    #only run once to create DB
    return create_engine("sqlite:///my_stock_data.db")




def fill_DB(stock_table_data,stock):
    # Create table if not exists
    stock_table_data.to_sql(stock, engine, if_exists="replace", index=False)

    #sorted descending by date

    stock_table_data = pd.read_sql_query("SELECT * FROM "+stock+" ORDER BY Date DESC", engine)


    # Create table if not exists
    stock_table_data.to_sql(stock, engine, if_exists="replace", index=False)
    



In [5]:
def calculateMA(stock_data):    
    # Calculate basic indicators
    stock_data["MA9"] = stock_data["Close"].rolling(window=9).mean()
    stock_data["MA21"] = stock_data["Close"].rolling(window=21).mean()
    stock_data["MA35"] = stock_data["Close"].rolling(window=35).mean()
    stock_data["MA50"] = stock_data["Close"].rolling(window=50).mean()
    stock_data["MA100"] = stock_data["Close"].rolling(window=100).mean()
    stock_data["MA200"] = stock_data["Close"].rolling(window=200).mean()
    return stock_data

In [6]:
def calculateVWAP(stock_data):
    # Calculate Typical Price
    stock_data['Typical Price'] = (stock_data['High'] + stock_data['Low'] + stock_data['Close']) / 3

    # Calculate VWAP
    stock_data['VWAP'] = (stock_data['Typical Price'] * stock_data['Volume']).cumsum() / stock_data['Volume'].cumsum()

    return stock_data

In [7]:
def calculateRSI(stock_data):
    # Calculate price differences
    delta = stock_data['Close'].diff()

    # Calculate gains and losses
    up = delta.where(delta > 0, 0)
    down = -delta.where(delta < 0, 0)

    # Calculate average gains and losses
    avg_gain = up.rolling(window=14).mean()
    avg_loss = down.rolling(window=14).mean().abs()

    # Calculate Relative Strength (RS) and RSI
    RS = avg_gain / avg_loss
    RSI = 100 - (100 / (1 + RS))

    stock_data['RSI'] = RSI
    return stock_data


In [8]:
    

def getRawData(stock_data):
    # Convert Date to datetime
    stock_data["Date"] = pd.to_datetime(stock_data.index)

    stock_data = calculateMA(stock_data)

    #stock_data = calculateVWAP(stock_data)

    stock_data = calculateRSI(stock_data)

    # Select relevant columns
    stock_DB_data = stock_data[["Date","Symbol", "Open", "High", "Low", "Close", "Volume", "MA9", "MA21", "MA35", "MA50", "MA100","MA200","RSI"]]
    return stock_DB_data


In [9]:
for stock in portfolio:
    engine = initialize_DB()
    stock_data = download_data(stock)
    stock_table_data = getRawData(stock_data)
    fill_DB(stock_table_data,stock)
    

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [32]:
# create method to use the existing database engine    
def analyse_data():
    engine = initialize_DB()
    overbought_days = pd.DataFrame()

    for stock in portfolio:
        # Example 1: Find all days with RSI above 70 (overbought)
        overbought_days = pd.concat([overbought_days, pd.read_sql_query(
        "SELECT DATE, SYMBOL, CLOSE, RSI FROM "+stock+" WHERE RSI > 70 AND DATE BETWEEN DATE('now', '-5 days') AND DATE('now')", engine)], ignore_index=True)

    # # Example 2: Calculate average volume for days with MA20 above Close
    # avg_vol_above_ma20 = pd.read_sql_query(
    #     "SELECT * FROM AAPL WHERE MA21 > Close", engine)
    
    overbought_days.to_sql("Overbought_stocks", engine, index=False, if_exists="replace")
    print(overbought_days)


In [33]:
analyse_data()

  Symbol
0  GOOGL
1  GOOGL
2    AMD
3    AMD
4    SPY
5    SPY


In [ ]:
#to kill or dispose the database
# engine.dispose()

In [ ]:
#delete table from DB
# pd.read_sql_query("DROP TABLE overbought_days", engine)

ResourceClosedError: This result object does not return rows. It has been closed automatically.